In [2]:
import sqlite3
import pandas as pd

# Connecting to the database
conn = sqlite3.connect('chinook.db')

#Q1  Customer Purchases Analysis
# Calculate total amount spent by each customer
query = """
SELECT c.CustomerId, c.FirstName || ' ' || c.LastName AS Name, 
       SUM(i.Total) AS TotalSpent
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId, c.FirstName, c.LastName
"""
df_spent = pd.read_sql_query(query, conn)

# Identify top 5 customers
top_5 = df_spent.sort_values(by='TotalSpent', ascending=False).head(5)

# Display results
print("Top 5 Customers by Total Spent:")
print(top_5)

Top 5 Customers by Total Spent:
    CustomerId                Name  TotalSpent
5            6         Helena Holý       49.62
25          26  Richard Cunningham       47.62
56          57          Luis Rojas       46.62
44          45     Ladislav Kovács       45.62
45          46       Hugh O'Reilly       45.62


In [3]:
#Q2 Album vs Individual Track Purchases
# Getting invoice line items with track and album info
query = """
SELECT DISTINCT i.CustomerId, t.AlbumId, t.TrackId
FROM invoice_items ii
JOIN tracks t ON ii.TrackId = t.TrackId
JOIN invoices i ON ii.InvoiceId = i.InvoiceId
"""
df_purchases = pd.read_sql_query(query, conn)

# Getting total tracks per album
query = """
SELECT AlbumId, COUNT(TrackId) AS TotalTracks
FROM tracks
GROUP BY AlbumId
"""
df_album_tracks = pd.read_sql_query(query, conn)

# Merging data to check purchases per customer
customer_purchases = df_purchases.groupby('CustomerId')['TrackId'].count().reset_index(name='TracksPurchased')
customer_albums = df_purchases.groupby(['CustomerId', 'AlbumId'])['TrackId'].count().reset_index(name='TracksPerAlbum')
customer_albums = customer_albums.merge(df_album_tracks, on='AlbumId')

# Determining preference
preferences = []
for customer_id in customer_purchases['CustomerId']:
    customer_data = customer_albums[customer_albums['CustomerId'] == customer_id]
    total_tracks = customer_purchases[customer_purchases['CustomerId'] == customer_id]['TracksPurchased'].iloc[0]
    album_tracks = customer_data['TotalTracks'].sum()
    
    if total_tracks < album_tracks and total_tracks > 0:
        preferences.append({'CustomerId': customer_id, 'Preference': 'Individual Tracks'})
    else:
        preferences.append({'CustomerId': customer_id, 'Preference': 'Full Albums'})

df_preferences = pd.DataFrame(preferences)

# Calculating percentages
total_customers = len(df_preferences)
ind_track_pct = (len(df_preferences[df_preferences['Preference'] == 'Individual Tracks']) / total_customers) * 100
full_album_pct = (len(df_preferences[df_preferences['Preference'] == 'Full Albums']) / total_customers) * 100

print("\nAlbum vs Individual Track Preferences:")
print(f"Percentage preferring Individual Tracks: {ind_track_pct:.2f}%")
print(f"Percentage preferring Full Albums: {full_album_pct:.2f}%")
print("\nPreference Summary:")
print(df_preferences['Preference'].value_counts())

# Close the database
conn.close()


Album vs Individual Track Preferences:
Percentage preferring Individual Tracks: 100.00%
Percentage preferring Full Albums: 0.00%

Preference Summary:
Preference
Individual Tracks    59
Name: count, dtype: int64
